In [ ]:
! pip install pytorch_pretrained_bert

In [ ]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
import time

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#if device != 'cpu':
  #torch.cuda.set_device(0)

In [ ]:
MODEL_PATH = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(MODEL_PATH)
model = BertForMaskedLM.from_pretrained(MODEL_PATH)
model.eval()
model.to(device)

100%|██████████| 407873900/407873900 [00:26<00:00, 15175730.15B/s]


BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
   

In [ ]:
def preprocess_text(text):

  text = str(text)
  target = '[PRED]'
  tokenized_text = tokenizer.tokenize(text)
  tokenized_text.append(target)
  masked_index = tokenized_text.index(target)
  tokenized_text[masked_index] = '[MASK]'
  indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
  seg_ids = [1] * len(tokenized_text)
  seg_ids[0] = 0
  seg_ids[1] = 0
  tokens_tensor = torch.tensor([indexed_tokens]).to(device)
  seg_tensor = torch.tensor([seg_ids]).to(device)

  return tokens_tensor, seg_tensor, masked_index

def predict(tk_tensor, seg_tensor,m_i,show_more=False):

  predictions = model(tk_tensor, seg_tensor)
  predicted_index = torch.argmax(predictions[0, m_i]).item()
  predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])
  
  if show_more:
    for i in range(10):
      predictions[0,m_i,predicted_index] = -11100000
      predicted_index = torch.argmax(predictions[0, m_i]).item()
      predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])
      print(predicted_token)

  return predicted_token  

def next_word(sentence, max_next_word=5):
  sentence = str(sentence)
  for n in range(max_next_word):
    tk,seg,m_i = preprocess_text(sentence)
    pred_tk = predict(tk,seg,m_i)
    
    sentence+= ' '+ pred_tk[0]
    print(sentence)

  

In [13]:
next_word('although he had already eaten a large meal, he was still very')

although he had already eaten a large meal, he was still very much
although he had already eaten a large meal, he was still very much he
although he had already eaten a large meal, he was still very much he was
although he had already eaten a large meal, he was still very much he was he
although he had already eaten a large meal, he was still very much he was he he
